# Maximum Error in CLIRAD-LW's Cooling Rate Estimates  
The error here is the amount of deviation from the corresponding line-by-line value, so it can be either positive or negative.  For cooling rates, we show the error in the troposphere and the stratosphere.  The maximum is taken over the layers in the respective sphere (troposphere or stratosphere) and over the mid-latitude summer, sub-arctic winter and tropical atmosphere profiles.  

In [1]:
%reload_ext autoreload
%autoreload 2

import pymongo
import pandas as pd

from climatools.cliradlw.utils import *
from climatools.atm.absorbers import *
from climatools.atm.regions import *
from climatools.dataset import *

import pprint
from IPython import display

In [2]:
COMMITNUMBER = '1013f91'

In [3]:
def strat_top(): return .05
def atm_top(): return .01

In [4]:
client = pymongo.MongoClient('localhost', 27017)

In [5]:
MOLECULES = ('h2o', 'co2', 'o3', 'n2o', 'ch4')
BANDS = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)
ATMPROS = dict([('mls', 294), ('saw', 257), ('trp', 300)])

In [6]:
def get_clirad_subparams(molecule=None, band=None):
    '''
    Return mls, saw and trp clirad parameter sub-dictionaries
    for a given molecule and clirad-lw spectral band.
    '''
    conc = nongreys_byband()[band][molecule]
    return {atmpro: {'commitnumber': COMMITNUMBER,
                     'band': [band],
                     'molecule': {molecule: conc},
                     'atmpro': atmpro,
                     'tsfc': tsfc}
            for atmpro, tsfc in ATMPROS.items()}

def to_lblnewparam(param=None):
    '''
    Convert clirad's input parameter dictionary 
    into something like lblnew's.
    '''
    band = param['band'][0]
    band = mapband_new2old()[band]
    atmpro = param['atmpro']
    tsfc = param['tsfc']
    
    p = {'band': band, 
         'atmpro': atmpro, 'tsfc': tsfc,
         'dv': .001, 'nv': 1000}
    
    if len(param['molecule']) > 1: p['molecule'] = param['molecule']
    else: 
        molecule, conc = list(param['molecule'].items())[0]
        p['molecule'] = molecule
        if conc == 'atmpro': p['conc'] = None
        else: p['conc'] = conc
    return p

def make_query(param=None):
    molecules = set(MOLECULES)
    q = {}
    for n, v in param.items():
        if n == 'molecule' and type(v) == dict:
            for mol, conc in v.items():
                q[f'param.molecule.{mol}'] = conc
            for mol in molecules - set(v.keys()):
                q[f'param.molecule.{mol}'] = {'$exists': 0}
        else:
            q[f'param.{n}'] = v
    return q

In [8]:
def get_maxerr_cool(molecule=None, band=None):
    '''
    Returns the maximum deviation from crd at the top and bottom
    of the atmosphere.
    '''
    tropos = tropopause_pressures()
    
    dclis = {atmpro: client.cliradnew.lw.find_one(make_query(param=p)) 
             for atmpro, p in get_clirad_subparams(molecule=molecule, band=band).items()}
    dclis = {atmpro: CliradnewLWModelData.from_mongodoc(doc) for atmpro, doc in dclis.items()}
    dclis = {atmpro: d.wgt_cool.sel(i=1).sum('band') for atmpro, d in dclis.items()}

    dcrds = {atmpro: client.lblnew.bestfit_lw.find_one(make_query(param=to_lblnewparam(p))) 
             for atmpro, p in get_clirad_subparams(molecule=molecule, band=band).items()}
    dcrds = {atmpro: LBLnewBestfitModelData.from_mongodoc(doc) for atmpro, doc in dcrds.items()}
    dcrds = {atmpro: d.crd_cool.sum('g') for atmpro, d in dcrds.items()}
    
    ddifs = {atmpro: dcli - dcrds[atmpro] for atmpro, dcli in dclis.items()}
    
    ddifs_tropo = [d.where(d.pressure > tropos[atmpro]) for atmpro, d in ddifs.items()]
    ddifs_strat = [d.where((tropos[atmpro] >= d.pressure) & (d.pressure > strat_top())) for atmpro, d in ddifs.items()]
    ddifs_topbt = [d.where((strat_top() >= d.pressure) & (d.pressure > atm_top())) for _, d in ddifs.items()]
    
    ddifs_tropo = [d.isel(pressure=abs(d.coolrg).argmax()) for d in ddifs_tropo]
    ddifs_strat = [d.isel(pressure=abs(d.coolrg).argmax()) for d in ddifs_strat]
    ddifs_topbt = [d.isel(pressure=abs(d.coolrg).argmax()) for d in ddifs_topbt]
    
    ddifs_tropo = sorted([d.coolrg for d in ddifs_tropo], key=lambda x: abs(x), reverse=True)
    ddifs_strat = sorted([d.coolrg for d in ddifs_strat], key=lambda x: abs(x), reverse=True)
    ddifs_topbt = sorted([d.coolrg for d in ddifs_topbt], key=lambda x: abs(x), reverse=True)

    return {'troposphere': float(ddifs_tropo[0]), 'stratosphere': float(ddifs_strat[0]),
            '.05 ~ .01 hPa': float(ddifs_topbt[0])}

In [9]:
def maxerr_cool_all():
    df = pd.DataFrame(index=MOLECULES, 
                      columns=pd.MultiIndex.from_product([BANDS, ('troposphere', 'stratosphere', '.05 ~ .01 hPa')], 
                                                         names=('band', 'region')))
    df.index.name = 'gas'

    for band, molconcs in nongreys_byband().items():
        for mol, conc in molconcs.items():
            for level, v in get_maxerr_cool(molecule=mol, band=band).items():
                df.loc[mol, (band, level)] = round(v, 5)

    df.fillna('', inplace=True)
    return df

In [10]:
df = maxerr_cool_all()
display.display(df.transpose())

gas                     h2o      co2       o3      n2o      ch4
band region                                                    
1    troposphere    0.05664                                    
     stratosphere   0.07188                                    
     .05 ~ .01 hPa -0.22914                                    
2    troposphere     0.0848                                    
     stratosphere  -0.02176                                    
     .05 ~ .01 hPa -0.08674                                    
3    troposphere    0.04771  -0.0082           0.00142         
     stratosphere  -0.00199 -0.09215          -0.00461         
     .05 ~ .01 hPa -0.00162 -0.15267            0.0042         
4    troposphere    0.05037  -0.0183                           
     stratosphere  -0.00257  0.70706                           
     .05 ~ .01 hPa -0.00124 -0.73417                           
5    troposphere    0.02228  0.01074                           
     stratosphere  -0.00079  0.08894                           
     .05 ~ .01 hPa -0.00022  -0.1418                           
6    troposphere    0.04972 -0.00051                           
     stratosphere  -0.00015 -0.00574                           
     .05 ~ .01 hPa   -3e-05 -0.00145                           
7    troposphere    0.01596 -0.00031 -0.00893                  
     stratosphere    -4e-05 -0.00622 -0.18143                  
     .05 ~ .01 hPa   -1e-05 -0.00158 -0.09091                  
8    troposphere     0.0248                                    
     stratosphere  -0.00055                                    
     .05 ~ .01 hPa -0.00014                                    
9    troposphere    0.02617                   -0.01056 -0.00407
     stratosphere  -0.00751                    0.02411  0.04036
     .05 ~ .01 hPa  0.00368                    0.02405  0.03968
10   troposphere   -0.02929                                    
     stratosphere   -0.0279                                    
     .05 ~ .01 hPa -0.03591                                    
11   troposphere    0.01406  0.00302                           
     stratosphere  -0.00147  0.06017                           
     .05 ~ .01 hPa  0.00065  0.04655

# Maximum Error in CLIRAD-LW's Flux Estimates  
The error here is the amount of deviation from the corresponding line-by-line value, so it can be either positive or negative.  For fluxes, we show the error in the upward flux at the top of the atmosphere and in the downward flux at the surface.  The maximum is taken over the mid-latitude summer, sub-arctic winter and tropical atmosphere profiles.

In [11]:
def get_maxerr_flux(molecule=None, band=None):
    '''
    Returns the maximum deviation from crd at the top and bottom
    of the atmosphere.
    '''
    tropos = tropopause_pressures()
    
    dclis = {atmpro: client.cliradnew.lw.find_one(make_query(param=p)) 
             for atmpro, p in get_clirad_subparams(molecule=molecule, band=band).items()}
    dclis = {atmpro: CliradnewLWModelData.from_mongodoc(doc) for atmpro, doc in dclis.items()}
    dclis = {atmpro: d.wgt_flux.sel(i=1).sum('band') for atmpro, d in dclis.items()}

    dcrds = {atmpro: client.lblnew.bestfit_lw.find_one(make_query(param=to_lblnewparam(p))) 
             for atmpro, p in get_clirad_subparams(molecule=molecule, band=band).items()}
    dcrds = {atmpro: LBLnewBestfitModelData.from_mongodoc(doc) for atmpro, doc in dcrds.items()}
    dcrds = {atmpro: d.crd_flux.sum('g') for atmpro, d in dcrds.items()}
    
    ddifs = {atmpro: dcli - dcrds[atmpro] for atmpro, dcli in dclis.items()}

    ddifs_toa = [float(d['flug'].isel(pressure=0).values) for atmpro, d in ddifs.items()]
    ddifs_sfc = [float(d['fldg'].isel(pressure=-1).values) for atmpro, d in ddifs.items()]
    
    maxdif_toa = sorted(ddifs_toa, key=lambda x: abs(x), reverse=True)[0]
    maxdif_sfc = sorted(ddifs_sfc, key=lambda x: abs(x), reverse=True)[0]
    return dict(TOA=maxdif_toa, SFC=maxdif_sfc)

In [12]:
def maxerr_flux_all():
    df = pd.DataFrame(index=MOLECULES, 
                      columns=pd.MultiIndex.from_product([BANDS, ('SFC', 'TOA')], 
                                                         names=('band', 'level')))
    df.index.name = 'gas'

    for band, molconcs in nongreys_byband().items():
        for mol, conc in molconcs.items():
            for level, v in get_maxerr_flux(molecule=mol, band=band).items():
                df.loc[mol, (band, level)] = round(v, 5)

    df.fillna('', inplace=True)
    return df

In [13]:
df = maxerr_flux_all()

In [14]:
#pd.set_option('display.max_columns', 50)
display.display(df.transpose())

gas             h2o      co2       o3      n2o      ch4
band level                                             
1    SFC   -0.00263                                    
     TOA    0.21081                                    
2    SFC    0.37487                                    
     TOA    0.32242                                    
3    SFC    0.41478   0.2371           0.01827         
     TOA   -0.09789 -0.12098          -0.05538         
4    SFC   -0.35914  0.09793                           
     TOA   -0.10343  0.41808                           
5    SFC    0.23032   0.4871                           
     TOA   -0.06182  0.36764                           
6    SFC    0.47091 -0.06794                           
     TOA    -0.0394  -0.0449                           
7    SFC    0.08334 -0.05404 -0.13916                  
     TOA   -0.04099 -0.03724  0.40439                  
8    SFC   -0.14038                                    
     TOA    -0.1106                                    
9    SFC   -0.13102                    -0.0549 -0.27249
     TOA   -0.32004                    0.22125 -0.03161
10   SFC    0.03607                                    
     TOA    0.14063                                    
11   SFC    0.52209  0.16514                           
     TOA      0.371  0.09987

# --

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')